# API Search Using a Leaflet Map
Search the API for imagery over an AOI selected from a map.

### Input Parameters 

In [ ]:
from os import environ

# Set API key (this should to be an environment variable)
api_key = environ['PLANET_API_KEY']

# A list of the types you are interested in.
item_types = ['PSScene3Band']

# Set max cloud coverage to 5%
max_cloud_coverage = .05

# The AOI you are intersted in, in GeoJSON format (San Francisco)
aoi_json = '''{
    "type": "Polygon",
        "coordinates": [
          [
            [
              -122.53944396972656,
              37.68979758086211
            ],
            [
              -122.35507965087889,
              37.68979758086211
            ],
            [
              -122.35507965087889,
              37.80924146650164
            ],
            [
              -122.53944396972656,
              37.80924146650164
            ],
            [
              -122.53944396972656,
              37.68979758086211
            ]
          ]
        ]
      }'''

# How many seach items to return
limit = 10

### Render Map, and search after AOI is selected
Render a map using ipyleaflet

In [ ]:
import json
from IPython.display import display, HTML
import pandas
from ipyleaflet import (
    Map,
    DrawControl
)
from planet import api

# Search items after AOI is selected
def handle_draw(self, action, geo_json):
    
    aoi = json.dumps(geo_json['geometry'])
    
    # Create client
    client = api.ClientV1(api_key)

    # Build a filter for the AOI
    query = api.filters.and_filter(
      api.filters.geom_filter(json.loads(aoi)),
      api.filters.range_filter('cloud_cover', lt=.05)
    )

    # Build search request
    request = api.filters.build_search_request(query, item_types)

    # Perform search
    items = client.quick_search(request)
    
    
    scenes = []
    for item in items.items_iter(limit=limit):
        props = {}
        props["id"] = item['id']
        props["acquired"] = item['properties']["acquired"]
        props["cloud_cover"] = item['properties']["cloud_cover"]
        props["geometry"] = item["geometry"]
        props["thumbnail"] = item["_links"]["thumbnail"]
        scenes.append(props)
       
    scenes_table = pandas.DataFrame(data=scenes)
    display(scenes_table)

# Render Map
leaflet_map = Map(zoom=14, center=(10.4891, -67.9996))
draw_controls =  DrawControl(
    rectangle={'shapeOptions': {'color': '#0000FF'}},
    polygon={'shapeOptions': {'color': '#0000FF'}},
    polyline={},
    circle={},
    edit=False,
    remove=False
)

draw_controls.on_draw(handle_draw)
leaflet_map.add_control(draw_controls)
leaflet_map